In [1]:
import intera_interface
import rospy
import json
import tf
from copy import deepcopy
from sawyer import transformations

In [2]:
rospy.init_node("sawyer_example")

In [3]:
effector = "right_hand"
world = "world"

In [45]:
tfl = tf.TransformListener(interpolate=True, cache_time=rospy.Duration(1000))
tfb = tf.TransformBroadcaster()

In [5]:
rs = intera_interface.RobotEnable(intera_interface.CHECK_VERSION)

In [6]:
rs.enable()

[INFO] [1554642635.526342]: Robot Enabled


In [7]:
limb = intera_interface.Limb('right')

# Compute sticks in cartesian space

In [80]:
# Base is ["lower"]["left"][0]
base_position = [-0.5, -0.68, 0.35]


In [81]:
# Starting from lower left (drawing direction)
# 
sticks = []
emotion = "hope"
y_interval = 0.015     # Interval of sticks in y direction
z_interval = 0.02      # Interval of sticks in z direction
group_interval = 0.02  # Interval between each group in y direction
num_groups = 7         # Number of stick groups per line
num_sticks = 4         # Number of sticks per group
height_stick = 0.04    # Length in z direction
num_height = 7         # Number of sticks in z direction (in height)

z_stick = base_position[2]
col = 0
for h in range(num_height):
    y_stick = base_position[1]
    for g in range(num_groups):
        for i in range(num_sticks):
            start_stick = [base_position[0], y_stick, z_stick]
            end_stick = [base_position[0], y_stick, height_stick + z_stick]
            sticks.append([start_stick, end_stick])

            tfb.sendTransform(sticks[-1][0], [0, 0, 0, 1], rospy.Time.now(), str(col), "world")
            tfb.sendTransform(sticks[-1][1], [0, 0, 0, 1], rospy.Time.now(), "e"+str(col), "world")
            y_stick += y_interval
            col += 1
        y_stick += group_interval
    z_stick += z_interval + height_stick
print(len(sticks), "sticks generated")

(196, 'sticks generated')


# Read approximate hand_to_stick transformation from tf

In [82]:
hope_lower_left = {u'right_j0': 1.611796875,
                   u'right_j1': -2.75679296875,
                   u'right_j2': -3.0216201171875,
                   u'right_j3': 1.296830078125,
                   u'right_j4': 1.161048828125,
                   u'right_j5': -1.98382421875,
                   u'right_j6': 1.0612880859375}

In [83]:
limb.move_to_joint_positions(hope_lower_left)
# Move to extreme lower left in JOINT space

In [84]:
CONSIDERED_STICK = 0

In [85]:
hand_T_stick = list(tfl.lookupTransform(str(CONSIDERED_STICK), effector, rospy.Time(0)))

In [86]:
hand_T_stick

[[0.14239612794977008, 0.01701314337065485, -0.07630844138995452],
 [0.5270677868272005,
  0.023880201241985057,
  -0.8461224120032248,
  0.07553904939543005]]

In [87]:
tfb.sendTransform(hand_T_stick[0], hand_T_stick[1], rospy.Time.now(), "hand_T_stick", str(CONSIDERED_STICK))

In [88]:
STICK_START = 0
STICK_END = 1
stick_T_world = [sticks[CONSIDERED_STICK][STICK_START], [0, 0, 0, 1]]

In [89]:
tfb.sendTransform(stick_T_world[0], stick_T_world[1], rospy.Time.now(), "stick_T_world", world)

In [93]:
hand_T_world = transformations.multiply_transform(stick_T_world, hand_T_stick)

In [94]:
tfb.sendTransform(hand_T_world[0], hand_T_world[1], rospy.Time.now(), "hand_T_world", world)

In [95]:
hand_T_world

[[-0.3576038720502299, -0.6629868566293452, 0.27369155861004546],
 [0.5270677868272005,
  0.023880201241985057,
  -0.8461224120032248,
  0.07553904939543005]]

# Get and dump IK for each stick

In [101]:
stick_ik = {"hope" : []}
for stick_id, stick in enumerate(sticks):
    hand_T_world = transformations.multiply_transform([stick[STICK_START], [0,0,0,1]], hand_T_stick)
    tfb.sendTransform(hand_T_world[0], hand_T_world[1], rospy.Time.now(), "ik", world)
    result = limb.ik_request(transformations.list_to_pose(hand_T_world))
    if result == False:
        raise RuntimeError("Can't compute stick start " + str(stick_id))
    hand_T_world_end = transformations.multiply_transform([stick[STICK_END], [0,0,0,1]], hand_T_stick)
    tfb.sendTransform(hand_T_world_end[0], hand_T_world_end[1], rospy.Time.now(), "eik", world)
    result_end = limb.ik_request(transformations.list_to_pose(hand_T_world_end))
    if result == False:
        raise RuntimeError("Can't compute stick end " + str(stick_id))
    stick_ik["hope"].append({"start": result, "end": result_end})
    #rospy.sleep(0.1)

In [104]:
with open("../config/sticks_target_ik.json", "w") as f:
    json.dump(stick_ik, f, indent=4)

# Overwrite other poses

In [108]:
# Pause pose
poses = {"pause": limb.joint_angles()}

with open("../config/poses.json", "w") as f:
    json.dump(poses, f, indent=4)

In [107]:
old = deepcopy(poses)

In [ ]:
# tests